# Programming Agent 2 – Refactor

Dieses Notebook ist so aufgeteilt, dass du:

1. **Parsing + Normalisierung** (aus der alten 2. Zelle) behältst  
2. **Gezieltes Einfügen unter `POUs` via `CreateChild`** (aus der alten 3. Zelle) nutzt  
3. Optional weiterhin **PLCopen-XML exportieren/importieren** kannst (Fallback / Debug)

> Tipp: Die COM-Cache-Löschung ist nur nötig, wenn `win32com` / TwinCAT Automation sich „verschluckt“.  


In [ ]:
# OPTIONAL: COM-Cache löschen (nur bei COM/Automation-Problemen nötig)
RUN_COM_CACHE_CLEANUP = False

if RUN_COM_CACHE_CLEANUP:
    import os, shutil, win32com
    gen_py_path = os.path.join(win32com.__gen_path__)
    print(f"Versuche COM-Cache zu löschen in: {gen_py_path}")

    if os.path.exists(gen_py_path):
        try:
            shutil.rmtree(gen_py_path)
            print("✅ Cache erfolgreich gelöscht! Bitte Kernel neu starten (Restart Kernel).")
        except PermissionError:
            print("❌ Zugriff verweigert. Bitte schließe alle Python/Jupyter Prozesse und lösche den Ordner manuell.")
        except Exception as e:
            print(f"❌ Fehler: {e}")
    else:
        print("ℹ️ Kein Cache gefunden (das ist gut).")


In [ ]:
import os
import time
import uuid
import xml.etree.ElementTree as ET
import win32com.client

# =========================
# 1) KONFIGURATION
# =========================

# TwinCAT / Visual Studio DTE ProgID (TwinCAT 3 XAE)
DTE_PROGID = "TcXaeShell.DTE.17.0"

# Solution (dein .sln)
SLN_PATH = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\TestProjektTwinCATEvents.sln"

# Quelle (TcPlcObject XML) – enthält <POU> + <Method> …
SOURCE_TC_OBJECT_XML = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\FB_GeneratedJob_01.xml"

# Ziel im TwinCAT-Projekt
PLC_PROJECT_NAME = "Proj1"        # so wie es im Solution Explorer heißt
TARGET_POU_NAME = "FB_GeneratedJob_01"  # nur Fallback; i.d.R. wird Name aus XML genommen

# IEC Language: Beckhoff erwartet i.d.R. "ST"
IECLANGUAGE_ST_STR = "ST"

# Verhalten
REPLACE_EXISTING = True           # existierendes FB mit gleichem Namen löschen
NORMALIZE_STATES = True           # Enum-State -> INT-State Normalisierung (aus alter 2. Zelle)
ENSURE_RETURN_VALUE = True        # Guard: Method Return setzen, falls vergessen

# OPTIONAL Debug/Fallback: PLCopen XML erzeugen (aus alter 2. Zelle)
WRITE_PLCOPEN_XML = False
OUT_DIR = r"D:\MA_Python_Agent\Notebooks\Generierte_PLCOpenXML"
OUT_FILE_NAME = "FB_GeneratedJob_01.plcopen.xml"
OUT_XML_PATH = os.path.join(OUT_DIR, OUT_FILE_NAME)

# PlcOpenImport Optionen (Fallback)
PLCIMPORTOPTIONS_REPLACE = 2


In [ ]:
# =========================
# 2) HELFER: Sicher schreiben (für optionalen PLCopen-Export)
# =========================
def safe_write_text_file(path: str, content: str, encoding="utf-8"):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    tmp_path = path + ".tmp"
    with open(tmp_path, "w", encoding=encoding, newline="\n") as f:
        f.write(content)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp_path, path)


# =========================
# 3) PARSE: TcPlcObject -> Declaration/Implementation/Methods
# =========================
def parse_tcplcobject_xml(tc_xml_path: str, default_pou_name: str):
    if not os.path.exists(tc_xml_path):
        raise FileNotFoundError(f"Quelle nicht gefunden: {tc_xml_path}")

    data = open(tc_xml_path, "rb").read()
    if data.startswith(b"\xef\xbb\xbf"):
        data = data[3:]

    root = ET.fromstring(data)
    ns = {"p": "http://www.plcopen.org/xml/tc6_0200"}

    pou = root.find("p:POU", ns)
    if pou is None:
        raise ValueError("Kein <POU> gefunden in TcPlcObject Datei.")

    pou_name = pou.attrib.get("Name", default_pou_name)

    decl_node = pou.find("p:Declaration", ns)
    pou_decl = (decl_node.text or "").strip()

    impl_node = pou.find("p:Implementation/p:ST", ns)
    pou_impl = (impl_node.text or "").strip()

    methods = []
    for m in pou.findall("p:Method", ns):
        m_name = m.attrib.get("Name", "Method")
        m_decl = (m.find("p:Declaration", ns).text or "").strip()
        m_code = (m.find("p:Implementation/p:ST", ns).text or "").strip()
        methods.append((m_name, m_decl, m_code))

    return pou_name, pou_decl, pou_impl, methods


def extract_method_return_type(method_decl: str) -> str:
    # Beispiel: "METHOD JobStart : BOOL"
    if ":" in method_decl:
        return method_decl.split(":")[-1].strip().split()[0].strip().upper()
    return "BOOL"


# =========================
# 4) TRANSFORM: State Enum -> INT State (aus alter 2. Zelle)
# =========================
def normalize_state_machine_code(code: str) -> str:
    if not code:
        return code

    out = code

    # Assignments
    out = out.replace("eState := RUNNING;", "eState := 1;")
    out = out.replace("eState := IDLE;", "eState := 0;")
    out = out.replace("eState := DONE;", "eState := 2;")
    out = out.replace("eState := ERROR;", "eState := 3;")
    out = out.replace("eState := ABORTED;", "eState := 4;")

    # CASE labels (indented / non-indented)
    out = out.replace("\n    IDLE:", "\n    0:")
    out = out.replace("\n    RUNNING:", "\n    1:")
    out = out.replace("\n    DONE:", "\n    2:")
    out = out.replace("\n    ERROR:", "\n    3:")
    out = out.replace("\n    ABORTED:", "\n    4:")

    out = out.replace("\nIDLE:", "\n0:")
    out = out.replace("\nRUNNING:", "\n1:")
    out = out.replace("\nDONE:", "\n2:")
    out = out.replace("\nERROR:", "\n3:")
    out = out.replace("\nABORTED:", "\n4:")

    return out


def ensure_method_returns(m_name: str, ret_type: str, code: str) -> str:
    """Setzt einen Rückgabewert, wenn im Code keiner gesetzt wird.
    Das ist hilfreich, weil TwinCAT sonst gerne mit 'not all code paths return a value' meckert.
    """
    if not code:
        code = ""

    if ret_type == "BOOL":
        if f"{m_name} :=" not in code:
            return code.rstrip() + f"\n{m_name} := TRUE;\n"
        return code

    if ret_type == "INT":
        if f"{m_name} :=" not in code:
            return code.rstrip() + f"\n{m_name} := eState;\n"
        return code

    # andere Types lassen wir unverändert
    return code


In [ ]:
# =========================
# 5) HELFER: DTE / Solution / TreeItem
# =========================
def _get_or_start_dte(progid: str):
    try:
        dte = win32com.client.GetActiveObject(progid)
    except Exception:
        dte = win32com.client.Dispatch(progid)
        dte.MainWindow.Visible = True
    dte.UserControl = True
    return dte


def _open_solution(dte, sln_path: str):
    if not os.path.exists(sln_path):
        raise FileNotFoundError(f"Solution nicht gefunden: {sln_path}")

    if (dte.Solution is None) or (not dte.Solution.IsOpen) or (dte.Solution.FullName.lower() != sln_path.lower()):
        print("Öffne Solution...")
        dte.Solution.Open(sln_path)

        for _ in range(60):
            if dte.Solution.IsOpen:
                break
            time.sleep(0.5)

    print("✅ Solution offen:", dte.Solution.FullName)


def _find_tsproj(dte):
    print("Suche TwinCAT Projekt (.tsproj)...")
    for i in range(1, dte.Solution.Projects.Count + 1):
        try:
            p = dte.Solution.Projects.Item(i)
            if p.FullName and p.FullName.lower().endswith(".tsproj"):
                print("✅ Gefunden:", p.Name)
                return p
        except Exception:
            continue
    return None


def _lookup_tree_item(sys_mgr, candidates):
    for c in candidates:
        try:
            return sys_mgr.LookupTreeItem(c)
        except Exception:
            continue
    return None


def _get_pous_folder(sys_mgr, plc_project_name: str):
    # Beckhoff hat je nach Sprache "Projekt" / "Project"
    candidates = [
        f"TIPC^{plc_project_name}^{plc_project_name} Projekt^POUs",
        f"TIPC^{plc_project_name}^{plc_project_name} Project^POUs",
    ]
    return _lookup_tree_item(sys_mgr, candidates)


In [ ]:
# =========================
# 6) OPTIONAL/FALLBACK: PLCopen <project> XML bauen + PlcOpenImport
# =========================
def build_plcopen_project_xml_minimal(pou_name: str, methods, pou_body_comment: str) -> str:
    """Minimaler PLCopen <project> Export (aus alter 2. Zelle).
    Nützlich als Debug/Fallback, wenn CreateChild nicht klappt.
    """
    PLC_NS = "http://www.plcopen.org/xml/tc6_0200"
    XHTML_NS = "http://www.w3.org/1999/xhtml"

    ET.register_namespace("", PLC_NS)
    ET.register_namespace("xhtml", XHTML_NS)

    project = ET.Element(f"{{{PLC_NS}}}project")

    fileHeader = ET.SubElement(project, f"{{{PLC_NS}}}fileHeader")
    fileHeader.set("companyName", "GeneratedByPython")
    fileHeader.set("productName", "TwinCAT PLC Control")
    fileHeader.set("productVersion", "3.5.x")
    fileHeader.set("creationDateTime", time.strftime("%Y-%m-%dT%H:%M:%S"))

    contentHeader = ET.SubElement(project, f"{{{PLC_NS}}}contentHeader")
    contentHeader.set("name", PLC_PROJECT_NAME)
    contentHeader.set("modificationDateTime", time.strftime("%Y-%m-%dT%H:%M:%S"))

    coordinateInfo = ET.SubElement(contentHeader, f"{{{PLC_NS}}}coordinateInfo")
    fbd = ET.SubElement(coordinateInfo, f"{{{PLC_NS}}}fbd")
    ET.SubElement(fbd, f"{{{PLC_NS}}}scaling", {"x": "1", "y": "1"})
    ld = ET.SubElement(coordinateInfo, f"{{{PLC_NS}}}ld")
    ET.SubElement(ld, f"{{{PLC_NS}}}scaling", {"x": "1", "y": "1"})
    sfc = ET.SubElement(coordinateInfo, f"{{{PLC_NS}}}sfc")
    ET.SubElement(sfc, f"{{{PLC_NS}}}scaling", {"x": "1", "y": "1"})

    types = ET.SubElement(project, f"{{{PLC_NS}}}types")
    ET.SubElement(types, f"{{{PLC_NS}}}dataTypes")
    pous = ET.SubElement(types, f"{{{PLC_NS}}}pous")

    pou = ET.SubElement(pous, f"{{{PLC_NS}}}pou")
    pou.set("name", pou_name)
    pou.set("pouType", "functionBlock")

    interface = ET.SubElement(pou, f"{{{PLC_NS}}}interface")
    localVars = ET.SubElement(interface, f"{{{PLC_NS}}}localVars")

    def add_var(name: str, typename: str):
        var = ET.SubElement(localVars, f"{{{PLC_NS}}}variable")
        var.set("name", name)
        t = ET.SubElement(var, f"{{{PLC_NS}}}type")
        ET.SubElement(t, f"{{{PLC_NS}}}{typename}")

    # Minimaler Satz an Variablen – passe das bei Bedarf an
    add_var("eState", "INT")
    add_var("bFinished", "BOOL")
    add_var("nCounter", "INT")
    add_var("fResult", "LREAL")

    body = ET.SubElement(pou, f"{{{PLC_NS}}}body")
    st = ET.SubElement(body, f"{{{PLC_NS}}}ST")
    xhtml = ET.SubElement(st, f"{{{XHTML_NS}}}xhtml")
    xhtml.text = pou_body_comment or "// FB Body"

    addData = ET.SubElement(pou, f"{{{PLC_NS}}}addData")
    for m_name, m_decl, m_code in methods:
        ret_type = extract_method_return_type(m_decl)
        m_code_norm = normalize_state_machine_code(m_code) if NORMALIZE_STATES else m_code
        m_code_norm = ensure_method_returns(m_name, ret_type, m_code_norm) if ENSURE_RETURN_VALUE else m_code_norm

        data = ET.SubElement(addData, f"{{{PLC_NS}}}data")
        data.set("name", "http://www.3s-software.com/plcopenxml/method")
        data.set("handleUnknown", "implementation")

        meth = ET.SubElement(data, f"{{{PLC_NS}}}Method")
        meth.set("name", m_name)
        meth.set("ObjectId", str(uuid.uuid4()))

        m_interface = ET.SubElement(meth, f"{{{PLC_NS}}}interface")
        returnType = ET.SubElement(m_interface, f"{{{PLC_NS}}}returnType")
        ET.SubElement(returnType, f"{{{PLC_NS}}}{ret_type}")

        m_body = ET.SubElement(meth, f"{{{PLC_NS}}}body")
        m_st = ET.SubElement(m_body, f"{{{PLC_NS}}}ST")
        m_xhtml = ET.SubElement(m_st, f"{{{XHTML_NS}}}xhtml")
        m_xhtml.text = m_code_norm

    instances = ET.SubElement(project, f"{{{PLC_NS}}}instances")
    ET.SubElement(instances, f"{{{PLC_NS}}}configurations")

    xml_bytes = ET.tostring(project, encoding="utf-8", xml_declaration=True)
    return xml_bytes.decode("utf-8")


def import_plcopen_into_twincat(sln_path: str, plcopen_xml_path: str):
    if not os.path.exists(sln_path):
        print(f"❌ Solution nicht gefunden: {sln_path}")
        return
    if not os.path.exists(plcopen_xml_path):
        print(f"❌ PLCopen Datei nicht gefunden: {plcopen_xml_path}")
        return

    print(f"Starte Automation Interface ({DTE_PROGID})...")

    try:
        dte = win32com.client.GetActiveObject(DTE_PROGID)
    except Exception:
        dte = win32com.client.Dispatch(DTE_PROGID)
        dte.MainWindow.Visible = True
    dte.UserControl = True

    _open_solution(dte, sln_path)

    tc_project = _find_tsproj(dte)
    if not tc_project:
        print("❌ Kein .tsproj gefunden.")
        return

    sys_mgr = tc_project.Object

    print("Starte PLCopen Import (PlcOpenImport)...")
    try:
        root_plc = sys_mgr.LookupTreeItem("TIPC")
        plc_project_item = None
        for i in range(1, root_plc.ChildCount + 1):
            child = root_plc.Child(i)
            try:
                if child.NestedProject:
                    plc_project_item = child.NestedProject
                    break
            except Exception:
                pass
        if not plc_project_item:
            print("❌ Kein SPS-Projekt gefunden.")
            return

        plc_project_item.PlcOpenImport(plcopen_xml_path, PLCIMPORTOPTIONS_REPLACE, "", True)
        print("✅ Import erfolgreich.")
    except Exception as e:
        print(f"❌ PlcOpenImport fehlgeschlagen: {e}")


In [ ]:
# =========================
# 7) HAUPTWEG: Direkt unter POUs erstellen (CreateChild)
# =========================
def create_or_update_fb_under_pous():
    # 1) Parse Source
    pou_name, pou_decl, pou_impl, methods = parse_tcplcobject_xml(SOURCE_TC_OBJECT_XML, default_pou_name=TARGET_POU_NAME)

    # 2) Optional Normalisierung
    if NORMALIZE_STATES:
        pou_impl = normalize_state_machine_code(pou_impl)
        normalized_methods = []
        for (m_name, m_decl, m_code) in methods:
            m_code = normalize_state_machine_code(m_code)
            normalized_methods.append((m_name, m_decl, m_code))
        methods = normalized_methods

    # 3) Optional Return-Guards
    if ENSURE_RETURN_VALUE:
        guarded_methods = []
        for (m_name, m_decl, m_code) in methods:
            ret_type = extract_method_return_type(m_decl)
            m_code = ensure_method_returns(m_name, ret_type, m_code)
            guarded_methods.append((m_name, m_decl, m_code))
        methods = guarded_methods

    # 4) Optional Debug/Fallback: PLCopen XML exportieren
    if WRITE_PLCOPEN_XML:
        xml_text = build_plcopen_project_xml_minimal(pou_name, methods, pou_impl)
        safe_write_text_file(OUT_XML_PATH, xml_text)
        print("✅ PLCopen XML geschrieben:", OUT_XML_PATH)

    # 5) TwinCAT Automation
    print("Starte Automation Interface...")
    dte = _get_or_start_dte(DTE_PROGID)
    _open_solution(dte, SLN_PATH)

    tc_project = _find_tsproj(dte)
    if not tc_project:
        raise RuntimeError("❌ Kein .tsproj gefunden.")

    sys_mgr = tc_project.Object

    # 6) POUs Ordner holen
    pous_item = _get_pous_folder(sys_mgr, PLC_PROJECT_NAME)
    if not pous_item:
        raise RuntimeError("❌ Ordner 'POUs' wurde nicht gefunden.")

    print("✅ Ordner POUs gefunden.")

    # 7) Existierendes FB löschen (optional)
    if REPLACE_EXISTING:
        existing_fb = None
        try:
            for i in range(1, pous_item.ChildCount + 1):
                c = pous_item.Child(i)
                if getattr(c, "Name", "") == pou_name:
                    existing_fb = c
                    break
        except Exception:
            pass

        if existing_fb:
            print(f"Altes Objekt '{pou_name}' gefunden -> Lösche es...")
            existing_fb.Delete()
            time.sleep(0.2)

    # 8) Function Block erstellen (Subtype 604)
    print(f"Erstelle Function Block '{pou_name}' direkt in POUs...")
    new_fb = pous_item.CreateChild(pou_name, 604, "", IECLANGUAGE_ST_STR)

    # Declaration / Implementation setzen
    try:
        new_fb.DeclarationText = pou_decl
    except Exception as e:
        print("⚠️ Konnte DeclarationText nicht setzen:", e)

    try:
        if pou_impl:
            new_fb.ImplementationText = pou_impl
    except Exception as e:
        print("⚠️ Konnte ImplementationText nicht setzen:", e)

    # 9) Methoden anlegen (Subtype 609)
    for (m_name, m_decl, m_code) in methods:
        ret_type = extract_method_return_type(m_decl)
        print(f"Erstelle Methode '{m_name}' (Return: {ret_type}) ...")

        # vInfo: [Language, ReturnType, AccessModifier]
        vinfo = [IECLANGUAGE_ST_STR, ret_type, "PUBLIC"]
        method_item = new_fb.CreateChild(m_name, 609, "", vinfo)

        try:
            method_item.DeclarationText = m_decl
        except Exception as e:
            print(f"⚠️ Konnte DeclarationText für Methode {m_name} nicht setzen:", e)

        try:
            method_item.ImplementationText = m_code
        except Exception as e:
            print(f"⚠️ Konnte ImplementationText für Methode {m_name} nicht setzen:", e)

    # 10) SaveAll
    try:
        dte.ExecuteCommand("File.SaveAll")
        print("✅ SaveAll ausgeführt.")
    except Exception:
        print("⚠️ Konnte SaveAll nicht ausführen (nicht kritisch).")

    print(f"✅ Fertig: '{pou_name}' ist jetzt unter POUs mit Methoden erstellt.")

# =========================
# AUSFÜHREN
# =========================
create_or_update_fb_under_pous()
